In [ ]:
#! pip install transformers datasets
! pip install tqdm
!pip install gliner

In [ ]:
import torch
from transformers import BertTokenizer, BertForSequenceClassification, Trainer, TrainingArguments
from datasets import load_dataset, DatasetDict
import pandas as pd
from sklearn.model_selection import train_test_split
#import torch
from transformers import BertTokenizer, BertForSequenceClassification, Trainer, TrainingArguments
from datasets import load_dataset
from sklearn.preprocessing import LabelEncoder

In [ ]:
import pandas as pd
from google.colab import drive

drive.mount('/content/drive')

csv_file_1 = '/content/drive/MyDrive/OsloTraineeship/embedding_training_data/Round1_train.csv'
csv_file_2 = '/content/drive/MyDrive/OsloTraineeship/embedding_training_data/Round3_train.csv'
csv_file_3 = '/content/drive/MyDrive/OsloTraineeship/embedding_training_data/2T_train.csv'
csv_file_4 = '/content/drive/MyDrive/OsloTraineeship/embedding_training_data/HT2_train.csv'
csv_file_5 = '/content/drive/MyDrive/OsloTraineeship/embedding_training_data/HT3_train.csv'
csv_file_6 = '/content/drive/MyDrive/OsloTraineeship/embedding_training_data/Round4_train.csv'

# Read the CSV files
df1 = pd.read_csv(csv_file_1)
df2 = pd.read_csv(csv_file_2)
df3 = pd.read_csv(csv_file_3)
df4 = pd.read_csv(csv_file_4)
df5 = pd.read_csv(csv_file_5)
df6 = pd.read_csv(csv_file_6)

dataset = pd.concat([df1, df2, df3, df4, df5, df6], axis=0)
dataset = dataset[dataset['desc'].notna()]


In [ ]:

result_df_path = '/content/drive/MyDrive/OsloTraineeship/embedding_training_data/embedding_result_df.csv'
result_df = pd.read_csv(result_df_path)

In [ ]:
result_df

In [ ]:
from gliner import GLiNER
from collections import Counter
import pandas as pd
from multiprocessing import Pool
from tqdm import tqdm

# Define a variable for the number of models
NUM_MODELS = 5

# Initialize the specified number of GLiNER model instances
models = [
    GLiNER.from_pretrained(
        "urchade/gliner_base"
    )
    for _ in range(NUM_MODELS)
]

# Define the batch processing function
def process_batch(args):
    model, batch, SEPARATOR, label_mapping, new_labels = args
    df = []
    pointer = 0

    # Prepare the text for the entire batch with a separator between each entry
    batch_text = SEPARATOR.join(batch['text'] + ' - ' + batch['desc'])

    # Predict entities for the entire batch using the configured tokenizer
    batch_entities = model.predict_entities(batch_text, new_labels, threshold=0.05)

    try:
        ner_type = []
        # Extract the first text entry before the separator
        if SEPARATOR in batch_text:
            text = batch_text[:batch_text.find(SEPARATOR)].split(" - ")[0]
        else:
            text = batch_text.split(" - ")[0]

        # Check if the text exists in the batch
        index = batch[batch['text'] == text].index
        if not index.empty:
            index = index[0]
            target = batch.iloc[index]['label']
            id = batch.iloc[index]['id']
            desc = batch_text[:batch_text.find(SEPARATOR)].split(" - ")[1]
        else:
            # Skip processing this batch if the index is not found
            return df

    except Exception as e:
        print(f"Exception occurred: {e}")
        return df

    for el in batch_entities:
        if (el['end'] - pointer) <= batch_text.find(SEPARATOR):
            ner_type.append(el['label'])
        else:
            counter = Counter(ner_type)
            try:
                most_common_element, element_count = counter.most_common(1)[0]
                new_row = {
                    'text': text,
                    'desc': desc,
                    'id': id,
                    'target': target,
                    'prediction': label_mapping.get(most_common_element, 'UNKNOWN')
                }
                #print(new_row)
                df.append(new_row)

                ner_type = []
                pointer += batch_text.find(SEPARATOR) + len(SEPARATOR)
                batch_text = batch_text[batch_text.find(SEPARATOR) + len(SEPARATOR):]

                if SEPARATOR in batch_text:
                    text = batch_text[:batch_text.find(SEPARATOR)].split(" - ")[0]
                else:
                    text = batch_text.split(" - ")[0]

                index = batch[batch['text'] == text].index
                if not index.empty:
                    index = index[0]
                    target = batch.iloc[index]['label']
                    id = batch.iloc[index]['id']
                    desc = batch_text[:batch_text.find(SEPARATOR)].split(" - ")[1]
                else:
                    # Skip processing this batch if the index is not found
                    return df

                if el['text'] in text:
                    ner_type.append(el['label'])
            except Exception as e:
                pointer += batch_text.find(SEPARATOR) + len(SEPARATOR)
                batch_text = batch_text[batch_text.find(SEPARATOR) + len(SEPARATOR):]
                continue

    return df

# Define the function for parallel batch processing
def parallel_batch_processing(dataset, models, batch_size, SEPARATOR, label_mapping, new_labels):
    results = []
    batches = []

    # Create batches of `batch_size` elements each
    for start in range(0, dataset.shape[0], batch_size):
        batch = dataset.iloc[start:start + batch_size].reset_index(drop=True)
        batches.append(batch)

    # Distribute tasks across the model instances
    tasks = [(models[i % len(models)], batches[i], SEPARATOR, label_mapping, new_labels) for i in range(len(batches))]

    # Use multiprocessing to process batches in parallel with the specified number of model instances
    with Pool(NUM_MODELS) as pool:
        for result in tqdm(pool.imap_unordered(process_batch, tasks), total=len(tasks), desc="Processing Batches"):
            results.extend(result)

    return pd.DataFrame(results)

# Define your variables
SEPARATOR = " ||| "
label_mapping = {
    "person": "PERS",
    "organization": "ORG",
    "location": "LOC",
    "film": "OTHERS",
    "others": "OTHERS",
    "videogames": "OTHERS",
    "date": "OTHERS",
    "galaxy": "OTHERS",
    "species": "OTHERS"
}
new_labels = [
    "person", "organization", "location", "others", "film", "videogames", "species", "date", "galaxy"
]
batch_size = 30

# Assuming `dataset` is your DataFrame, process the dataset in parallel using the specified number of model instances
result_df = parallel_batch_processing(dataset, models, batch_size, SEPARATOR, label_mapping, new_labels)


In [ ]:
# Save the DataFrame to a CSV file
result_df.to_csv('/content/drive/MyDrive/OsloTraineeship/embedding_training_data/embedding_result_df.csv', index=False)

print("DataFrame saved ")

# Reading the resulting file

In [11]:
import pandas as pd

# Load the CSV file
df = pd.read_csv('./data/embedding_training_data/embedding_result_df.csv')


# Display the first few rows of the data
len(df[df['target'] != df['prediction']])/len(df)


0.17647379614986095